In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import numpy as np
from torch.utils.data import Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/5180-project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json
  
# Opening JSON file
f = open('/content/drive/My Drive/5180-project/train.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
# for i in data:
#     print(i)
  
# # Closing file
# f.close()

In [5]:
data

[{'question': "Who are the individuals who have held the title 'Minister for Magic'?",
  'SPARQL': 'SELECT ?individual WHERE { ?individual hp:title hp:Minister_for_magic. }'},
 {'question': 'What is the blood status of Cornelius Fudge and Pius Thicknesse?',
  'SPARQL': 'SELECT ?individual ?blood WHERE { ?individual a hp:Individual_ ; hp:blood ?blood . FILTER (?individual IN (hp:Cornelius_fudge, hp:Pius_thicknesse)) }'},
 {'question': 'Who are the members of the Edgecombe family?',
  'SPARQL': 'SELECT ?member WHERE { hp:Edgecombe_family hp:members ?member .}'},
 {'question': 'Who are the enemies of the International Confederation of Wizards?',
  'SPARQL': 'SELECT ?enemy WHERE { hp:International_confederation_of_wizards hp:enemies ?enemy .}'},
 {'question': 'What is the location of London and who are its residents?',
  'SPARQL': 'SELECT ?location ?resident WHERE { hp:London hp:location ?location . OPTIONAL {?location hp:residents ?resident .}}'},
 {'question': 'Which individuals have a b

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# data

In [8]:
y = []
for i in range(len(data)):
  y.append(0)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33, random_state=42)

In [10]:
class Dataset(Dataset):
    """   
    fun exercise: does it help if the result is asked to be produced in reverse order?
    """

    def __init__(self, data):
        self.data = data #data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # given a problem index idx, first recover the associated a + b
        # idx = self.ixes[idx]
        render = self.data[idx] 

        return {"input_ids":tokenizer(render['question'])["input_ids"], "labels":tokenizer(render['SPARQL'])["input_ids"]}

In [53]:
model_checkpoint = "/content/drive/My Drive/5180-project/hp_v3.4"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint,max_length = 1024)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer(["0", "01", "20"])

In [54]:
batch_size = 64
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=1e-3,
    save_total_limit=3,
    num_train_epochs=30,
    gradient_accumulation_steps = 3,
    predict_with_generate=True,
    push_to_hub=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [55]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=Dataset(X_train),
    eval_dataset=Dataset(X_test),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,No log,11.768638
1,No log,8.787288
3,No log,6.787366
3,No log,5.390290
4,No log,4.174825
6,No log,3.624648
6,No log,2.877405
7,No log,2.245879
9,No log,2.085821
9,No log,1.873800


TrainOutput(global_step=30, training_loss=3.041009775797526, metrics={'train_runtime': 99.3379, 'train_samples_per_second': 63.118, 'train_steps_per_second': 0.302, 'total_flos': 101711067494400.0, 'train_loss': 3.041009775797526, 'epoch': 22.5})

In [56]:
test = Dataset(X_test)
count = 0
for index in range(len(test)):
  device = 'cuda'
  preds = model.generate(input_ids = torch.tensor(test[index]["input_ids"]).to(device).view(1,-1))
  print(tokenizer.decode(np.array(preds.cpu()[0]))[7:-4])
  print(tokenizer.decode(np.array(test[index]["labels"]))[3:-4])
  print('------------------------------------------------------')
  # print(preds)
  # count+=1.0*(tokenizer.decode(np.array(preds.cpu()[0])) == tokenizer.decode(np.array(test[index]["labels"])))
  # if index%10==0:
    # print(tokenizer.decode(np.array(preds.cpu()[0])),tokenizer.decode(np.array(test[index]["labels"])))
# count/len(test)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


SELECT?name WHERE { hp:Cornelius_fudges_wand hp:name?name.}
SELECT?individualName WHERE { hp:Chief_warlock_of_the_wizengamot_in_1544 hp:name?individualName.}
------------------------------------------------------
SELECT?name WHERE { hp:Cornelius_fudges_wand hp:name?name.}
SELECT?location WHERE { hp:Whitehall hp:location?location.}
------------------------------------------------------
SELECT?name WHERE { hp:Wizengamot hp:name?name.}
SELECT?member WHERE { hp:Edgecombe_family hp:members?member.}
------------------------------------------------------
SELECT?name WHERE { hp:Cornelius_fudges_wand hp:name?name.}
SELECT DISTINCT?gender WHERE {?individual hp:loyalty hp:British_ministry_of_magic.?individual hp:gender?gender. }
------------------------------------------------------
SELECT?name WHERE { hp:Wizengamot hp:name?name.}
SELECT?member WHERE { hp:Edgecombe_family hp:members?member.}
------------------------------------------------------
SELECT?name WHERE { hp:Cornelius_fudges_wand hp:nam

In [57]:
train = Dataset(X_train)
count = 0
for index in range(len(train)):
  device = 'cuda'
  preds = model.generate(input_ids = torch.tensor(train[index]["input_ids"]).to(device).view(1,-1))
  print(tokenizer.decode(np.array(preds.cpu()[0]))[7:-4])
  print(tokenizer.decode(np.array(train[index]["labels"]))[3:-4])
  print('------------------------------------------------------')

SELECT?name WHERE { hp:British_ministry_of_magic hp:name?name.}
SELECT?affiliation?intention WHERE { hp:Department_of_magical_accidents_and_catastrophes hp:affiliation?affiliation. hp:Department_of_magical_accidents_and_catastrophes hp:intentions?intention.}
------------------------------------------------------
SELECT?name WHERE { hp:British_ministry_of_magic hp:name?name.}
SELECT?leader WHERE { hp:Educational_office hp:leader?leader.}
------------------------------------------------------
SELECT?name WHERE { hp:Wizengamot hp:name?name.}
SELECT?member WHERE { hp:Edgecombe_family hp:members?member.}
------------------------------------------------------
SELECT?name WHERE { hp:Wizengamot hp:name?name.}
SELECT?individualName WHERE { hp:Edgecombe_family hp:loyalty?loyalty ; hp:name hp:Edgecombe_family.?individual hp:loyalty?loyalty ; hp:name?individualName.}
------------------------------------------------------
SELECT?name WHERE { hp:Cornelius_fudges_wand hp:name?name.}
SELECT?name WHERE

In [42]:
preds.cpu()[0]

tensor([    2,     0, 49179, 17487,   605,   463,  1215, 13650, 29919, 25522,
        29886,    35, 43078,   523,  6125,  1215,   506,  1906,  5641,     2])

In [43]:
tokenizer.decode(np.array(preds.cpu()[0]))[7:-4]

'</s><s>SELECT?wand_name WHERE { hp:Cornelius_fudges</s>'

In [44]:
tokenizer.decode(np.array(test[index]["labels"]))[3:-4]

'<s>SELECT?wand_name WHERE { hp:Cornelius_fudges_wand hp:name?wand_name.}</s>'